# A MultiClass Neural Network #

Let's try coding a Multiclass neural network from scratch

In [1]:
# import useful librairies
import numpy as np

In [2]:
# Let's make a multilabel dataset

# Initialize randomness
np.random.seed(42)

# Our dataset is made of coded numbers
zero = np.array([[1],[1],[1],[0],[1],[1],[1]])
one = np.array([[0],[0],[0],[0],[0],[1],[1]])
two = np.array([[0],[1],[1],[1],[1],[1],[0]])
three = np.array([[0],[0],[1],[1],[1],[1],[1]])
four = np.array([[1],[0],[0],[1],[0],[1],[1]])
five = np.array([[1],[0],[1],[1],[1],[0],[1]])
six = np.array([[1],[1],[1],[1],[1],[0],[1]])
seven = np.array([[0],[0],[1],[0],[0],[1],[1]])
eight = np.array([[1],[1],[1],[1],[1],[1],[1]])
nine = np.array([[1],[0],[1],[1],[1],[1],[1]])

# A function to decode our number
def decode_num(x):
    r = x
    r = np.insert(r, 0, [0])
    r = np.insert(r, -2, [0])
    r = r.reshape(9,1)
    r = np.append(r[:3], np.append(r[3:6], r[6:], axis = 1), axis = 1)
    stringa = ''
    stringb = ''
    stringc = ''
    for i,y in enumerate(r):
        for j,v in enumerate(y):
            if i == 0:
                if v == 1 and j == 1:
                    stringa = stringa + '_'
                else:
                    stringa = stringa + ' '
            if i == 1:
                if v == 1 and j == 1:
                    stringb = stringb + '_'
                elif v == 1:
                    stringb = stringb + '|'
                else:
                    stringb = stringb + ' '
            if i == 2:
                if v == 1 and j == 1:
                    stringc = stringc + '_'
                elif v == 1:
                    stringc = stringc + '|'
                else:
                    stringc = stringc + ' '
    print(stringa)
    print(stringb)
    print(stringc)

# Making m examples from our coded numbers
m = 20
# Choosing randomly the numbers for m examples
pre_dataset = np.random.randint(0, 10, (1, m))

# Setting labels
Y = np.zeros(0)
for v in pre_dataset[0]:
    Y = np.append(Y, [0 for x in range(v)] +[1] + [0 for x in range(9-v)])
Y = Y.reshape(m,10)

# Setting inputs
numbers = [zero, one, two, three, four, five, six, seven, eight, nine]
X = np.zeros(0)
for v in pre_dataset[0]:
    X = np.append(X, [numbers[v]])
X = X.reshape(m,7)

# Let's check if everything works find
for i, v in enumerate(Y):
    decode_num(X[i])
    print('label : ' , np.where(v == 1)[0])

 _ 
|_ 
|_|
label :  [6]
 _ 
 _|
 _|
label :  [3]
 _ 
  |
  |
label :  [7]
   
|_|
  |
label :  [4]
 _ 
|_ 
|_|
label :  [6]
 _ 
|_|
 _|
label :  [9]
 _ 
 _|
|_ 
label :  [2]
 _ 
|_ 
|_|
label :  [6]
 _ 
  |
  |
label :  [7]
   
|_|
  |
label :  [4]
 _ 
 _|
 _|
label :  [3]
 _ 
  |
  |
label :  [7]
 _ 
  |
  |
label :  [7]
 _ 
 _|
|_ 
label :  [2]
 _ 
|_ 
 _|
label :  [5]
   
|_|
  |
label :  [4]
   
  |
  |
label :  [1]
 _ 
  |
  |
label :  [7]
 _ 
|_ 
 _|
label :  [5]
   
  |
  |
label :  [1]


In [3]:
# Activation function
def activation(x, function = 'sigmoid'):
    if function == 'sigmoid':
        return 1/(1+np.exp(-x))
    else:
        raise NotImplementedError

In [17]:
# The neural network

# Initialisation
# the architecture (hidden layers)
N = [3]
m = X.shape[0]

# Adding the input and output layers
N = [X.shape[1]] + N + [Y.shape[1]]

# Check X, Y and N
print('X shape : ', X.shape)
print('Y shape : ', Y.shape)
print('Archi : ', N)

# Weights and bias
np.random.seed(42)
W = [np.random.randn(N[r], N[r+1]) for r in range(len(N)-1) ]
b = [np.zeros((1, N[r+1])) for r in range(len(N)-1)]

# Hyperparameters
lr = 0.5
epoch = 100
costs = []

# The training
for i in range(epoch):
    print('LOOP : ', i)
    A = [X]
    # Forward
    for l in range(len(N)-1):
#         print('LAYER : ', l+1)
        # the last layer
        if l == len(N) - 2:
            # compute the softmax
            Z = np.dot(A[l], W[l]) + b[l] 
            Ypred = np.exp(Z) / np.sum(np.exp(Z))
        else:
#             print(A[l].shape)
#             print(W[l].shape)
            Z = np.dot(A[l], W[l]) + b[l]
            A.append(activation(Z))
#         print(A[l].shape)
    
#     print('Ypred : ', Ypred.shape)
    # Cost
    L = - np.sum(Y * np.log(Ypred)) / m
    print('Loss : ', L)
    costs.append(L)
    accuracy  = (1 - np.mean(np.abs(Ypred - Y))) * 100
    print('Accuracy : ', accuracy)
    
    # Backward
    dw = []
    db = []
    dz = Ypred - Y # (m 10)
    dw.insert(0, np.dot(A[-1].T, dz) / m) # (m p)
    db.insert(0, np.sum(dz, axis = 0, keepdims=True) / m) # (1 p)
    for l in reversed(range(len(N) - 2)):
        dz = np.dot(dz, W[l+1].T) * (A[l+1] * (1 - A[l+1]))
        dw.insert(0, np.dot(A[l].T, dz) / m)
        db.insert(0, np.sum(dz, axis = 0, keepdims=True) / m)
    
#     for v in dw:
#         print('dw : ', v.shape)
    
    # Update
    for l in range(len(N)-1):
#         print('update : ',W[l].shape)
#         print(dw[l].shape)
        W[l] = W[l] - lr * dw[l]
        b[l] = b[l] - lr * db[l]
        
accuracy  = (1 - np.mean(np.abs(Ypred - Y))) * 100
print(accuracy)
    
        
    

X shape :  (20, 7)
Y shape :  (20, 10)
Archi :  [7, 3, 10]
LOOP :  0
Loss :  5.487801580455904
Accuracy :  89.59976162257247
LOOP :  1
Loss :  5.42928721093797
Accuracy :  89.60674295635303
LOOP :  2
Loss :  5.377661363910407
Accuracy :  89.61403585679248
LOOP :  3
Loss :  5.332276741925242
Accuracy :  89.62166879176897
LOOP :  4
Loss :  5.292728892604638
Accuracy :  89.6296652592581
LOOP :  5
Loss :  5.258810579251351
Accuracy :  89.63804224846481
LOOP :  6
Loss :  5.230476111207694
Accuracy :  89.64680623348708
LOOP :  7
Loss :  5.207807964267763
Accuracy :  89.65594745854294
LOOP :  8
Loss :  5.1909811994937325
Accuracy :  89.66543352952114
LOOP :  9
Loss :  5.180223560905816
Accuracy :  89.67520367057728
LOOP :  10
Loss :  5.175771379119773
Accuracy :  89.68516530879644
LOOP :  11
Loss :  5.177823781377303
Accuracy :  89.69519467707777
LOOP :  12
Loss :  5.186500007456279
Accuracy :  89.70514260676865
LOOP :  13
Loss :  5.201806179503447
Accuracy :  89.71484549478615
LOOP :  14
Los